In [1]:
import numpy as np
import pandas as pd

In [2]:
np.__version__

'1.19.2'

In [249]:
class Tensor:
    def __init__(self,array,grad=None,requires_grad=False):
        """
        array: numpy array
        grad: dic
        """
        self.array=array
        self.requires_grad=requires_grad
        
        if requires_grad:
            name=id(self) #use this to identify the tensor
            #name of the parameter
            self.grad={name: self.make_grad()}
        else:
            self.grad={'none':0}
        if grad is not None:
            self.grad=grad

    def make_grad(self,):
        shape=self.array.shape
        Kron=1
        for d in shape:
            ID=np.identity(d)
            Kron=np.tensordot(Kron,ID,axes=0)
        new_shape=[i for i in range(0,2*len(shape),2)]
        new_shape+=[i for i in range(1,2*len(shape),2)]
        Kron=Kron.transpose(new_shape)

        return Kron
    @property
    def shape(self):
        return self.array.shape
    @property
    def ndim(self):
        return self.array.ndim
    @property
    def grad_shape(self):
        return self.grad.shape

    def __add__(self,x):
        result=self.array+x.array

        for w in self.grad:
            if w not in x.grad:
                x.grad[w]=0
        for w in x.grad:
            if w not in self.grad:
                self.grad[w]=0
        grad={}
        for w in self.grad:
            grad[w]=self.grad[w]+x.grad[w]

        return Tensor(result,grad=grad)

    def __mul__(self,x):
        if isinstance(x,int) or isinstance(x,float):
            result=x*self.array
            grad={}
            for w in self.grad:
                grad[w]=x*self.grad[w]

        if isinstance(x,Tensor):
            result=np.tensordot(self.array,x.array,axes=([-1],[0]))

            for w in self.grad:
                if w not in x.grad:
                    x.grad[w]=0
            for w in x.grad:
                if w not in self.grad:
                    self.grad[w]=0

            grad={}
            for w in self.grad:
                if x.grad[w] is 0:
                    grad1=0
                else:
                    grad1=np.tensordot(self.array,x.grad[w],axes=([-1],[0]))
                    
                if self.grad[w] is 0:
                    grad2=0
                else:
                    i=len(self.array.shape)
                    grad2=np.tensordot(self.grad[w],x.array,axes=([i-1],[0]))
                    n1=self.grad[w].ndim
                    n2=self.array.ndim
                    n3=x.array.ndim
                    r1=[j for j in range(n2-1)]+[j for j in range(n1-1,n1+n3-2)]
                    r2=[j for j in range(n2-1,n1-1)]
                    grad2=grad2.transpose(r1+r2)
                
                grad[w]=grad1+grad2

        return Tensor(result,grad=grad)
    """
    def __rmul__(self, x):
        if isinstance(x,int) or isinstance(x,float):
            result=x*self.array
            grad=x*self.grad
            return Tensor(result,grad=grad)
        if isinstance(x,Tensor):
            return self.__mul__(x)
    """
    def sum(self,axis):
        result=self.array.sum(axis=axis)
        grad={}
        for w in self.grad:
            if self.grad[w] is not 0:
                grad[w]=self.grad[w].sum(axis=axis)
            else:
                grad[w]=0
        return Tensor(result,grad=grad)

    def __repr__(self):

        return f'Tensor({self.array},dtype {self.array.dtype},requires_grad={self.requires_grad})'

In [62]:
class Sigmoid:
    """
    returns: Tensor with gradients
    """
    def __call__(self,x):

        out=1/(1+np.exp(-x.array))
        grad={}
        for w in x.grad:
            if x.grad[w] is not 0:
                i=x.ndim
                l=x.grad[w].ndim
                expand=tuple([k for k in range(i,l)])
                grad_func=self.grad(x)
                grad_func=np.expand_dims(grad_func,axis=expand)
                grad[w]=grad_func*x.grad[w]
            else:
                grad[w]=0

        return Tensor(out,grad=grad)

    def grad(self,x):
        den=(1+np.exp(-x.array))*(1+np.exp(-x.array))
        gd=np.exp(-x.array)/den

        return gd

class Exp:
    """
    returns: Tensor with gradients
    """
    def __call__(self,x):

        out=np.exp(x.array)
        grad=self.grad(x)*x.grad

        return Tensor(out,grad=grad) 
    
    def grad(self,x):
        gd=np.exp(x.array)
        return gd


# Feed-foward neural network

In [250]:
class LinearLayer:
    def __init__(self,in_dim,out_dim,bias=True):
        self.in_dim=in_dim
        self.out_dim=out_dim

        weight_,bias_=self.init_params()

        self.weight=Tensor(weight_,requires_grad=True)
        self.bias=Tensor(bias_,requires_grad=True)
    
    def __call__(self,x):
        """
        x: Tensor [batch,in_dim]
        """
        out=x*self.weight+self.bias

        return out

    def init_params(self):
        weight=np.random.rand(self.in_dim,self.out_dim)
        bias=np.random.rand(1,self.out_dim)

        return weight, bias

class FeedForward:

    def __init__(self,input_dim,hidden_dim,out_dim=1):
        self.in_layer=LinearLayer(input_dim,hidden_dim)
        self.out_layer=LinearLayer(hidden_dim,out_dim)
        self.sig=Sigmoid()

    def __call__(self,x):
        out=self.in_layer(x)
        out=self.sig(out)
        out=self.out_layer(out)
        #we assume a two class classification problem
        out=self.sig(out)

        return out

In [6]:
from sklearn.datasets import load_breast_cancer

In [7]:
data=load_breast_cancer()

In [136]:
x=data['data']
y=data['target']

In [137]:
x.shape

(569, 30)

In [256]:
x_tensor=Tensor(x,requires_grad=False)
y_tensor=Tensor(y,requires_grad=False)

In [255]:
model=FeedForward(30,10)

In [257]:
out=model(x_tensor)

In [262]:
L=out.sum(axis=0)

In [264]:
L.grad.keys()

dict_keys(['none', 140522626309648, 140522626272144, 140522626269392, 140522626270480])